> First time use: follow instructions in the README.md file in this directory.



# Receitas

Várias receitas para usar Timelink com notebooks

[Sumário]

---

# Receipts

Receipts for using Timelink with notebooks

[summary]


---
## Setup

### Basic setup

Create a `TimelinkNotebook` instance.

This will provide an interface to the 
various `Timelink` functions.



In [14]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()


### Changing the default values

When called with no parameters `TimelinkNotebook` 
assumes a certain directory layout containing the notebook
and from that layout autoconfigures various parameters.

The default layout:

* **project-directory**
    * **database** (database related filed)
    * **notebooks**  _(directory with the current notebook)_ 
    * **kleio**  (source files in kleio format)
    * **identifications** (files with identification information)

Based on this layout `TimelinkNotebook` assumes a few default values:

* **database name**
    * the name of the *project directory* is used as the **database name**, sanitized to produce a valid database name
      (hifens and spaces are replaced with _ ).
* **database type**
  * the type of the database is assumed to be **sqlite**  and a **sqlite** database is created in 
      *database/sqlite*
    * the `db_type` parameter can be used to set the database to `postgres`. 
* **Kleio server home**, working directory for the _Kleio server_ is assumed to be the **project directory**
  * The _Kleio server_ is an application
    that processes transcriptions of historical _Kleio_ notation and generates data in a format
    that can be imported to the database.

These default values can be changed with parameters when creating the `TimelinkNotebook`  instance:

```python
tlnb = TimelinkNotebook(
    project_name="my-project",
    project_home="~/projects/m-project",
    db_type="postgres",
    db_name="my_project_db",
    kleio_image="timelinkserver/kleio-server",
    kleio_version="12.0.0",
    postgres_image="postgres",
    postgres_version="latest",
    sqlite_dir="~/databases",
    sql_echo=True
)
```

### Examining the current configuration

In [15]:

tlnb.print_info()


Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: gTMTcxSPsD96MhYi9Eu3cxVXmaBYht6k
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: pensive_moore
Kleio version requested: latest
Kleio server version: 12.4.567 (2024-02-07 13:02:03)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


### Listing available databases

In [16]:
pd = tlnb.get_postgres_databases()
print(f"Postgres databases: {pd}")
sd = tlnb.get_sqlite_databases()
print(f"Sqlite databases: {sd}")


Postgres databases: ['timelink', 'tests']
Sqlite databases: ['../database/sqlite/dehergne_locations.sqlite']


### Database status (table row count)

In [17]:
tlnb.table_row_count_df()

,table,count
0,acts,29
1,aregisters,1
2,attributes,25482
3,class_attributes,70
4,classes,14
5,entities,31624
6,geoentities,359
7,goods,0
8,kleiofiles,31
9,objects,0


## Dealing with kleio files

### List Kleio files available

In [18]:
kleio_files = tlnb.get_kleio_files()
kleio_files.head()

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/dehergne-a.cli,dehergne-a.cli,2023-11-29 02:16:33.838462+00:00,V,2023-10-13 04:46:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:22:59.712358,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
1,sources/dehergne-b.cli,dehergne-b.cli,2023-11-29 02:16:33.847301+00:00,V,2023-10-13 04:47:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:23:11.907971,/rest/reports/sources/dehergne-b.rpt,/rest/exports/sources/dehergne-b.xml
2,sources/dehergne-c.cli,dehergne-c.cli,2023-11-29 02:16:33.858180+00:00,V,2023-10-13 04:47:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:23:29.418688,/rest/reports/sources/dehergne-c.rpt,/rest/exports/sources/dehergne-c.xml
3,sources/dehergne-d.cli,dehergne-d.cli,2023-11-29 02:16:33.873745+00:00,V,2023-10-13 04:47:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:23:36.819380,/rest/reports/sources/dehergne-d.rpt,/rest/exports/sources/dehergne-d.xml
4,sources/dehergne-e.cli,dehergne-e.cli,2023-11-29 02:16:33.879442+00:00,V,2023-10-13 04:47:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:23:38.526859,/rest/reports/sources/dehergne-e.rpt,/rest/exports/sources/dehergne-e.xml


Show only translation and import status

In [19]:
kleio_files[["name","status","import_status"]]

,name,status,import_status
0,dehergne-a.cli,V,I
1,dehergne-b.cli,V,I
2,dehergne-c.cli,V,I
3,dehergne-d.cli,V,I
4,dehergne-e.cli,V,I
5,dehergne-f.cli,V,I
6,dehergne-g.cli,V,I
7,dehergne-h.cli,V,I
8,dehergne-i.cli,V,I
9,dehergne-j.cli,V,I


Show translation and import errors

In [20]:
kleio_files[["name","errors","warnings","import_errors","import_warnings"]]

,name,errors,warnings,import_errors,import_warnings
0,dehergne-a.cli,0,0,0,0
1,dehergne-b.cli,0,0,0,0
2,dehergne-c.cli,0,0,0,0
3,dehergne-d.cli,0,0,0,0
4,dehergne-e.cli,0,0,0,0
5,dehergne-f.cli,0,0,0,0
6,dehergne-g.cli,0,0,0,0
7,dehergne-h.cli,0,0,0,0
8,dehergne-i.cli,0,0,0,0
9,dehergne-j.cli,0,0,0,0


### Translation and Import reports

Translation reports.

Pass a dataframe with Kleio files and a row number to get the translation report

In [21]:
rpt=tlnb.get_translation_report(kleio_files,rows=1)
print(rpt)

KleioTranslator - server version 12.0 - build 541 (MHK: 2235/master) (2023-07-04 04:34:23, MHK: 06/08/2023 04:15 )
13-10-2023 4-46

Processing data file dehergne-b.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 187
Obs: 
** Processing source fonte$dehergne-b
7: lista$dehergne-notices-b
*** End of File

Line 358 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-antonio-de-abreu) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 838 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-alessandro-cicero) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 1458 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-louis-porquet) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.

Structure file: /kleio-home/sources/dehergne/structures/sources.str
Structure processing report: /kleio-home/sources/deherg

Or use the file name

In [22]:
file = kleio_files.iloc[0].rpt_url
print(file)
rpt=tlnb.get_translation_report(file)
print(rpt)

/rest/reports/sources/dehergne-a.rpt
KleioTranslator - server version 12.0 - build 541 (MHK: 2235/master) (2023-07-04 04:34:23, MHK: 06/08/2023 04:15 )
13-10-2023 4-46

Processing data file dehergne-a.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 193
Obs: 
** Processing source fonte$dehergne-a
7: lista$dehergne-notices-a
*** End of File

Line 541 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-belchior-miguel-carneiro-leitao) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 814 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-jean-regis-lieou) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 876 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-alessandro-cicero) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.

Structure file: /kleio-home/sources/dehergne/structures/sources.str
S

Import report

In [23]:
rpt = tlnb.get_import_rpt(kleio_files,rows=1)
print(rpt)

No errors
No warnings



# Atualizar base de dados

Atualiza traduções de fontes e importa quando tradução não tem erros.

---

# Update database

Updates source translations and imports into database sources with no errors.

In [35]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

Show imported files status

In [36]:
imported_files_df = tlnb.get_import_status()
imported_files_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   path                30 non-null     object             
 1   name                30 non-null     object             
 2   size                30 non-null     int64              
 3   directory           30 non-null     object             
 4   modified            30 non-null     datetime64[ns, UTC]
 5   modified_iso        30 non-null     datetime64[ns, UTC]
 6   modified_string     30 non-null     object             
 7   qtime               30 non-null     datetime64[ns, UTC]
 8   qtime_string        30 non-null     object             
 9   source_url          30 non-null     object             
 10  status              30 non-null     object             
 11  translated          30 non-null     datetime64[ns, UTC]
 12  translated_string   30 non-null     ob

Check the import status of the translated files
```python
    I = "I" # imported
    E = "E" # imported with error
    W = "W" # imported with warnings no errors
    N = "N" # not imported
    U = "U" # translation updated need to reimport

``` 

In [38]:
imported_files_df[["import_status","import_errors","import_warnings","name","imported","path"]].sort_values("name")

,import_status,import_errors,import_warnings,name,imported,path
0,I,0,0,dehergne-a.cli,2024-02-07 03:22:59.712358,sources/dehergne-a.cli
1,I,0,0,dehergne-b.cli,2024-02-07 03:23:11.907971,sources/dehergne-b.cli
2,I,0,0,dehergne-c.cli,2024-02-07 03:23:29.418688,sources/dehergne-c.cli
3,I,0,0,dehergne-d.cli,2024-02-07 03:23:36.819380,sources/dehergne-d.cli
4,I,0,0,dehergne-e.cli,2024-02-07 03:23:38.526859,sources/dehergne-e.cli
5,I,0,0,dehergne-f.cli,2024-02-07 03:23:49.669086,sources/dehergne-f.cli
6,I,0,0,dehergne-g.cli,2024-02-07 03:24:00.428726,sources/dehergne-g.cli
7,I,0,0,dehergne-h.cli,2024-02-07 03:24:04.137667,sources/dehergne-h.cli
8,I,0,0,dehergne-i.cli,2024-02-07 03:24:06.156498,sources/dehergne-i.cli
9,I,0,0,dehergne-j.cli,2024-02-07 03:24:08.424135,sources/dehergne-j.cli


## Todo

This as data frame in a single function
* TimelinkNotebook.translate([files_df,rows=List[int],status="T"])
* TimelinkNotebook.import(files_df, rows=List[int])



# Obter dados
---

# Getting data

### Procurar pessoa

---
### Search for people

#### Search persons by name

In [39]:
from timelink.api.models import Person
from sqlalchemy import select

show_only=10

with tlnb.db.session() as session:
    stmt = select(Person).where(Person.name.like('% Aboim'))
    print(stmt)
    persons = session.execute(stmt).scalars().all()
    print()
    for person in persons[:show_only]:
        print(person.id, person.name,person.sex)

SELECT persons.id, entities.id AS id_1, entities.class, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE persons.name LIKE :name_1



### Search other Entities

#### get the Entity classes in the database

In [40]:
from sqlalchemy import select, func
from timelink.api.models import Entity

models = tlnb.db.get_models_ids()
models

['attribute',
 'relation',
 'act',
 'source',
 'person',
 'good',
 'object',
 'class',
 'geoentity',
 'rgeoentity',
 'robject',
 'rperson',
 'rentity',
 'aregister',
 'entity']

#### Get columns of an entity type

In [41]:
table = tlnb.db.get_table("entity")
print(table.name)
list(table.columns)

entities


[Column('id', String(), table=<entities>, primary_key=True, nullable=False),
 Column('class', String(), table=<entities>, nullable=False),
 Column('inside', String(), ForeignKey('entities.id'), table=<entities>),
 Column('the_order', Integer(), table=<entities>),
 Column('the_level', Integer(), table=<entities>),
 Column('the_line', Integer(), table=<entities>),
 Column('groupname', String(), table=<entities>),
 Column('updated', DateTime(), table=<entities>, default=CallableColumnDefault(<function datetime.utcnow at 0x133fc00d0>)),
 Column('indexed', DateTime(), table=<entities>)]

#### Search any entity type

## IMPROVE

In [30]:
from timelink.api.models import Entity
from sqlalchemy import select, func

Geoentity = tlnb.db.get_model("geoentity")
stmt = select(Geoentity).where(Geoentity.name.like('H%'))
print(stmt)
with tlnb.db.session() as session:

    result = session.execute(stmt).scalars().all()
    for row in result:
        print()
        print(row.the_type,row.name,row.id,row.obs)
        print(row.to_kleio())

SELECT geoentities.id, entities.id AS id_1, entities.class, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, geoentities.name, geoentities.obs, geoentities.the_type 
FROM entities JOIN geoentities ON entities.id = geoentities.id 
WHERE geoentities.name LIKE :name_1

geo2 Hangchou deh-r1644-hangchou None


geo2$deh-r1644-hangchou/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q4970"/16440000
  atr$activa/sim/16110000
  atr$residencia-missao/Jesuíta/16110000
  geo3$deh-r1644-fuyang/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q1011103"/16440000
    atr$activa/sim/16420000
  geo3$deh-r1644-jenho/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/16080000

geo2 Huchow deh-r1644-huchow None
geo2$deh-r1644-huchow/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q42664"/16440000
  geo3$deh-r1644-tehtsing/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/1620

geo3 Hungtang deh-r1644-hungtang None
geo3$deh-r1644-hungtang/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  a

#### Get person by id

Show a single person or entity in Kleio notation

In [45]:
p = tlnb.db.get_person('deh-michele-ruggieri')
print(p.to_kleio())

AttributeError: 'NoneType' object has no attribute 'to_kleio'

### Show other type of entities by id in Kleio

In [ ]:
from timelink.api.models import Entity

ent = tlnb.db.get_entity("deh-r1644-chekiang")
print(ent.to_kleio())

geo1$deh-r1644-chekiang/type=geoentity
  rel$function-in-act/geo1/deh-chre-1644/16440000
  geo2$deh-r1644-hangchou/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    atr$activa/sim/16110000
    atr$residencia-missao/Jesuíta/16110000
    geo3$deh-r1644-fuyang/type=geoentity
      rel$function-in-act/geo3/deh-chre-1644/16440000
      atr$activa/sim/16420000
    geo3$deh-r1644-jenho/type=geoentity
      rel$function-in-act/geo3/deh-chre-1644/16440000
      atr$activa/sim/16080000
  geo2$deh-r1644-chuchow/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    atr$activa/sim/16130000
  geo2$deh-r1644-huchow/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    geo3$deh-r1644-tehtsing/type=geoentity
      rel$function-in-act/geo3/deh-chre-1644/16440000
      atr$activa/sim/1620
  geo2$deh-r1644-kashing/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    geo3$deh-r1644-kaosham/type=geoentity
      rel$function-in-act/geo3



###  Obter um dataframe a partir de atributos

---


###  Get a Dataframe from attributes


#### Exemplo: Faculdade, data de entrada e data de saída e grau dos naturais de Coimbra

In [46]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='person',
                    the_type='naturalidade',
                    the_value='Coimbra',
                    more_info=['name','sex'],
                    name_like='% Aboim',
                    more_cols=['faculdade','uc.entrada','uc.saida'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

AttributeError: 'NoneType' object has no attribute 'info'

In [ ]:
df.head(5)

,name,sex,naturalidade,naturalidade.date,naturalidade.obs,faculdade,faculdade.date,faculdade.obs,uc.entrada,uc.entrada.date,uc.entrada.obs,uc.saida,uc.saida.date,uc.saida.obs
id,,,,,,,,,,,,,,
140349,António de Aboim,m,Coimbra,1566-12-20,None,Cânones,1566-12-20,None,1566-12-20,1566-12-20,None,1574-07-24,1574-07-24,None
140367,Manuel de Vargas de Aboim,m,Coimbra,20200211,None,Cânones,20200211,None,0000-00-00,20200211,None,0000-00-00,20200211,None


#### obter attributos de outras entidades

In [ ]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='geoentity',
                    more_info=['name'],
                    the_type='activa',
                    the_value='sim',
                    more_cols=['residencia-missao'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253 entries, deh-r1644-anhai to deh-r1701-yunnanfu
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    253 non-null    object
 1   activa                  253 non-null    object
 2   activa.date             253 non-null    object
 3   activa.obs              5 non-null      object
 4   residencia-missao       88 non-null     object
 5   residencia-missao.date  88 non-null     object
 6   residencia-missao.obs   11 non-null     object
dtypes: object(7)
memory usage: 15.8+ KB


In [ ]:
df.head(30)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None
deh-r1644-chala,Chala,sim,1610,None,NaN,NaN,NaN
deh-r1644-changchow-fou,Changchow,sim,1643,None,NaN,NaN,NaN
deh-r1644-changshu,Changshu,sim,1623,None,Jesuíta,1635,"R 1635 «Cham Xo», cf AHSI 28 (1951) 311-312"
deh-r1644-chengting,Chengting,sim,1621,None,NaN,NaN,NaN




###  Remover colunas sem valores

---



###  Remove empty columns

In [ ]:
df.dropna(how='all', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, deh-r1644-anhai to deh-r1644-yunnan
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    165 non-null    object
 1   activa                  165 non-null    object
 2   activa.date             165 non-null    object
 3   activa.obs              4 non-null      object
 4   residencia-missao       38 non-null     object
 5   residencia-missao.date  38 non-null     object
 6   residencia-missao.obs   4 non-null      object
dtypes: object(7)
memory usage: 10.3+ KB


In [ ]:
df.head(5)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None


In [ ]:
ids = df.index.unique()
for id in ids[:5]:
    ent = tlnb.db.get_entity(id)
    print(ent.to_kleio())
    print()

geo3$deh-r1644-anhai/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1634

geo3$deh-r1644-bankao/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1636

geo2$deh-r1644-cantao/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$activa/sim/1555
  atr$residencia-missao/Jesuíta/1580
  atr$residencia-missao/Franciscanos/0000
  atr$residencia-missao/Dominicanos/0000
  geo3$deh-r1644-tungkun/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
  geo3$deh-r1644-quon-yao/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/1621
  geo3$deh-r1644-lampacao/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/1535

geo3$deh-r1644-chala/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1610

geo2$deh-r1644-changchow-fou/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$activa/sim/1643



## Contagens

---

## Counting



###  Contagem de atributos a partir de uma tabela em memória

---

###  Count attributes from an existing dataframe



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253 entries, deh-r1644-anhai to deh-r1701-yunnanfu
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    253 non-null    object
 1   activa                  253 non-null    object
 2   activa.date             253 non-null    object
 3   activa.obs              5 non-null      object
 4   residencia-missao       88 non-null     object
 5   residencia-missao.date  88 non-null     object
 6   residencia-missao.obs   11 non-null     object
 7   id                      253 non-null    object
dtypes: object(8)
memory usage: 17.8+ KB


In [13]:
# create a column with the index values which are the id numbers
df['id'] = df.index.values

col = 'residencia-missao' # subotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = df.groupby(col).agg({'residencia-missao.date':['count','min','max']}

                                )
df_totals#.sort_values('id',ascending= False).head(30)

residencia-missao.date                    
                                      count       min       max
residencia-missao                                              
Agostinhos                                5      1701      1701
Dominicanos                              15      0000      1701
Franciscanos                             21      0000      1701
Jesuíta                                  37      1580      1701
Lazaristas                                1  17020400  17020400
Missões Estrangeiras                      2  17020400  17021100
Missões estrangeiras                      6      1701      1701
Propaganda                                1      1703      1703


### Contagens na base de dados

Quando o atributo tem muitos valores e não é necessário
ter todas as pessoas em memória: contagem feita na base de dados

---

### Counting directly in the database
When there are many values and it is not
necessary to have all the people in memory:
count directly in the database.




In [ ]:
from timelink.pandas import attribute_values

df_totals = attribute_values('grau',db=tlnb.db)


In [ ]:
df_totals.head(10)


,count,date_min,date_max
value,,,
Bacharel,170,1554-07-19,1912-08-03
Formatura,153,1574-07-24,1905-06-19
Licenciado,33,1574-06-03,1886-02-27
Bacharel em Artes,25,1574-03-14,1766-07-19
Doutor,11,1560-12-22,1887-11-27
Licenciado em Artes,5,1574-05-15,1738-06-17
Mestre,2,1710-10-05,1768-10-23


#### Filtrar por datas

Para evitar remissivas com data zero

---

#### Filter by dates

Avoid cross-references with zero date

##ERROR

In [ ]:
df_totals = attribute_values('grau',dates_between=('1535','1919'),db=tlnb.db)

In [ ]:
df_totals.head(10)

,count,date_min,date_max
value,,,
Bacharel,170,1554-07-19,1912-08-03
Formatura,153,1574-07-24,1905-06-19
Licenciado,33,1574-06-03,1886-02-27
Bacharel em Artes,25,1574-03-14,1766-07-19
Doutor,11,1560-12-22,1887-11-27
Licenciado em Artes,5,1574-05-15,1738-06-17
Mestre,2,1710-10-05,1768-10-23


## Visualizar registos

---

## View records





### Visualizar uma pessoa

---

### View a person


#### Atributos de uma pessoa numa tabela, uma linha por attributo

---

#### Person attributes in a dataframe, one line per attribute

In [ ]:
import pandas as pd
from timelink.pandas import group_attributes as person_attributes

pd.set_option('display.max_rows',1000)

id = '140349'
pdf = person_attributes([id],db=tlnb.db)  # note id in a list
pdf[['date','type','value','attr_obs']].sort_values(['date','type'])

,date,type,value,attr_obs
id,,,,
140349,1566-12-20,faculdade,Cânones,None
140349,1566-12-20,faculdade.ano,Cânones.1566,None
140349,1566-12-20,naturalidade,Coimbra,None
140349,1566-12-20,naturalidade.ano,Coimbra.1566,None
140349,1566-12-20,nome,António de Aboim,None
140349,1566-12-20,nome-geografico,Coimbra,None
140349,1566-12-20,nome-geografico.ano,Coimbra.1566,None
140349,1566-12-20,nome.apelido,Aboim,None
140349,1566-12-20,nome.primeiro,António,None


#### Atributos de uma pessoa numa tabela, attributos em colunas

---

#### Person attributes in a dataframe, attributes in columns

In [30]:
# Get list of people with with a certain value in a specific attribute

df = entities_with_attribute(
                    entity_type='person',
                    the_type='uc.entrada',  # we need a base attribute
                    more_info=['name','sex'],
                    more_cols=['instituta','matricula.faculdade','matricula.ano'],
                    filter_by=[140349],
                    db=tlnb.db,
                    sql_echo=False)
view_cols = ['name','matricula.ano.date','matricula.ano','matricula.ano.obs']
df[view_cols].sort_values('matricula.ano.date')

,name,matricula.ano.date,matricula.ano,matricula.ano.obs
id,,,,
140349,António de Aboim,1571-07-20,Cânones.1571,20.07.1571
140349,António de Aboim,1573-10-07,Cânones.1573,07.10.1573
140349,António de Aboim,1574-07-24,Cânones.1574,24.07.1574


#### Examinar potenciais duplicados

---

#### Examine potentital duplicates

In [15]:
from timelink.pandas import display_group_attributes
pd.set_option('display.max_rows',250)

no_show=['código-de-referência','data-do-registo','url','faculdade.ano','naturalidade.ano',
         'matricula-faculdade.ano','nome-apelido','nome-primeiro','nome-geografico.ano',
         'grau.ano','matricula-outra.ano','nome-geografico','instituta.ano']

dup_ids = ['234295','234710',]  # Alexandre Metelo de

display_group_attributes(dup_ids,
                             header_cols=['uc-entrada','naturalidade','faculdade','nome-pai'],
                             exclude_attributes=no_show,
                             sort_attributes=['date','type','value'],
                             cmap_name='Pastel1')

,id,uc-entrada,naturalidade,faculdade,nome-pai
0,234295,1704-11-07,Marialva,Cânones,nan
1,234710,1705-10-24,Marialva,Matemática,Manuel Cardoso Metelo


,date,id,type,value,attr_obs
0,1704-11-07,234295,faculdade,Cânones,Cânones
1,1704-11-07,234295,instituta,1704-11-07,07.11.1704 1704-11-07
2,1704-11-07,234295,naturalidade,Marialva,None
3,1704-11-07,234295,nome,Alexandre Metelo de Sousa,None
4,1704-11-07,234295,uc-entrada,1704-11-07,None
5,1704-11-07,234295,uc-entrada.ano,1704,None
6,1705-10-24,234710,faculdade,Matemática,Matemática
7,1705-10-24,234295,matricula-faculdade,Cânones,24.10.1705
8,1705-10-24,234710,matricula-faculdade,Matemática,24.10.1705
9,1705-10-24,234710,naturalidade,Marialva,None


#### Notação Kleio

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation

See [Kleio notation](README_kleio.md)

#### Notação Kleio directamente da base de dados

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation directly from database

See [Kleio notation](README_kleio.md)

In [33]:
from timelink.mhk.models.person import Person

with tlnb.db.session() as session:

    p: Person = session.query(Person).order_by(Person.id).first()
    k = p.to_kleio()
    print(p.to_kleio())


n$António Pinto Abadeço/m/id=140337/obs="""
      """

                  Id: 140337
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000001

                  Nome        : António Pinto Abadeço
                  Data inicial: 1705-10-01
                  Data final  : 1710-10-01
                  Filiação: António Pinto

                  Naturalidade: Abrantes
                  Faculdade: Cânones

                  Matrícula(s): 01.10.1705
                  01.10.1706
                  01.01.1707
                  01.10.1708
                  01.10.1709
                  01.10.1710
                  Instituta:
                  Bacharel: 28.06.1709
                  Formatura:
                  Licenciado:
                  Doutor:

                  Outras informações:
              """
  """
  rel$function-in-act/n/António Pinto Abadeço/auc-alumni-A-140337-140771/20200211
  atr$código-de-referência/""PT/AUC/ELU/UC-AUC/B/001-001/A/000001""/2021-05-17
  atr$data-